In [3]:
import pandas as pd
import html5lib
import bs4 #beautiful soup
import numpy as np
from xml.sax import ContentHandler, parse


In [4]:
# Reference https://goo.gl/KaOBG3 and 
# http://stackoverflow.com/questions/33470130/read-excel-xml-xls-file-with-pandas
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)


In [107]:
#get tips data
excelHandler = ExcelHandler()#instantiate object
parse('https://www.federalreserve.gov/econresdata/researchdata/feds200805.xls', excelHandler)


In [108]:
#create dateframe
tips_data = pd.DataFrame(
    pd.DataFrame(excelHandler.tables[0][18:]).values[1:,1:],#data
    index=pd.to_datetime(pd.DataFrame(excelHandler.tables[0][18:]).values[1:,0]),#row labels; dates
    columns=pd.DataFrame(excelHandler.tables[0][18:]).values[0,1:]#column labels
    )
#clean up column names:
tips_data.columns=tips_data.columns.str.strip()#.str.lower()
tips_data=tips_data.filter(regex='TIPSY')
#convert data to numeric
for c in tips_data.columns:
    tips_data[c] = pd.to_numeric(tips_data[c],errors='coerce')

tips_data = tips_data.sort_index()

tips_data.head()



,TIPSY02,TIPSY03,TIPSY04,TIPSY05,TIPSY06,TIPSY07,TIPSY08,TIPSY09,TIPSY10,TIPSY11,...,BKEVENF20,BKEVEN1F4,BKEVEN1F9,BKEVEN5F5,BETA0,BETA1,BETA2,BETA3,TAU1,TAU2
1999-01-04,NaN,NaN,NaN,3.9244,3.9369,3.9310,3.9170,3.9002,3.8832,3.8673,...,2.1389,0.6546,2.1212,1.5634,3.6688,-1.7499,3.0498,NaN,1.7262,NaN
1999-01-05,NaN,NaN,NaN,3.9385,3.9525,3.9475,3.9340,3.9175,3.9005,3.8845,...,2.2029,0.6754,2.1493,1.5840,3.6826,-1.7540,3.0624,NaN,1.7479,NaN
1999-01-06,NaN,NaN,NaN,3.8522,3.8357,3.8182,3.8026,3.7893,3.7783,3.7691,...,2.1431,0.9434,2.1055,1.6518,3.6753,-1.8079,2.7590,NaN,1.0862,NaN
1999-01-07,NaN,NaN,NaN,3.8995,3.8827,3.8550,3.8282,3.8051,3.7859,3.7698,...,2.2173,0.8224,2.3029,1.8021,3.6067,-9.9883,11.8613,NaN,0.9583,NaN
1999-01-08,NaN,NaN,NaN,3.9043,3.8985,3.8771,3.8525,3.8293,3.8089,3.7913,...,2.2188,0.8888,2.3839,1.8609,3.6057,-4.4381,6.1063,NaN,1.2291,NaN


In [111]:
tips_data.describe()

,TIPSY02,TIPSY03,TIPSY04,TIPSY05,TIPSY06,TIPSY07,TIPSY08,TIPSY09,TIPSY10,TIPSY11,...,BKEVENF20,BKEVEN1F4,BKEVEN1F9,BKEVEN5F5,BETA0,BETA1,BETA2,BETA3,TAU1,TAU2
count,3095.000000,3095.000000,3095.00000,4342.000000,4342.000000,4342.000000,4342.000000,4342.000000,4342.000000,4342.000000,...,4321.000000,4321.000000,4321.000000,4321.000000,4345.000000,4345.000000,4345.000000,3098.000000,4345.000000,3098.000000
mean,0.123697,0.263000,0.42840,1.270058,1.411061,1.530710,1.632997,1.720768,1.796173,1.860930,...,2.475859,2.133576,2.790282,2.575427,4.586231,-8.484971,-209.204647,286.592559,2.762315,17.859749
std,1.458861,1.337625,1.25679,1.572145,1.514368,1.456187,1.400332,1.348295,1.300768,1.257967,...,0.615518,0.403595,0.435706,0.360728,5.774358,66.852410,5896.251531,6981.168418,2.607693,11.347393
min,-2.202000,-2.067000,-1.89280,-1.724000,-1.538900,-1.357800,-1.188300,-1.016600,-0.862700,-0.724600,...,-0.742800,0.485900,1.305300,1.336100,0.000000,-1331.498657,-183813.517600,-40412.588798,0.100000,0.436327
25%,-1.055050,-0.933900,-0.63060,-0.039475,0.152350,0.319150,0.456200,0.585925,0.685825,0.786400,...,2.155300,1.863000,2.582500,2.416500,1.461966,-5.962488,-5.911649,-14.691889,1.190363,5.333289
50%,-0.126100,0.053600,0.26500,1.243950,1.400750,1.542150,1.657600,1.761250,1.848500,1.921300,...,2.553900,2.246100,2.824000,2.612300,3.124400,-2.902191,-2.681086,-2.620848,1.998098,15.767847
75%,0.983150,1.132350,1.28770,2.382450,2.387700,2.415550,2.443800,2.469575,2.495175,2.521750,...,2.814100,2.433400,3.088000,2.808300,4.066600,-0.694500,1.652600,6.019598,4.220037,30.000000
max,5.479800,4.486200,4.01380,4.378400,4.385600,4.383800,4.380900,4.377400,4.371100,4.366800,...,3.873300,2.814800,3.923900,3.625300,25.000000,1696.301410,40412.078681,183812.729900,33.069000,30.000000


In [109]:
#get nominal data
excelHandler = ExcelHandler()#instantiate object
parse('http://www.federalreserve.gov/econresdata/researchdata/feds200628.xls', excelHandler)


In [110]:
#create dateframe
nominal_data = pd.DataFrame(
    pd.DataFrame(excelHandler.tables[0][9:]).values[1:,1:],#data
    index=pd.to_datetime(pd.DataFrame(excelHandler.tables[0][9:]).values[1:,0]),#row labels; dates
    columns=pd.DataFrame(excelHandler.tables[0][9:]).values[0,1:]#column labels
    )
#clean up column names:
nominal_data.columns=nominal_data.columns.str.strip()#.str.lower()
nominal_data=nominal_data.filter(regex='SVENY')

#convert data to numeric
for c in nominal_data.columns:
    nominal_data[c] = pd.to_numeric(nominal_data[c],errors='coerce')

nominal_data = nominal_data.sort_index()

nominal_data.head()

,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10,...,SVENF30,SVEN1F01,SVEN1F04,SVEN1F09,BETA0,BETA1,BETA2,BETA3,TAU1,TAU2
1961-06-14,2.9825,3.3771,3.5530,3.6439,3.6987,3.7351,3.7612,NaN,NaN,NaN,...,NaN,3.8067,3.9562,NaN,3.917606,-1.277955,-1.949397,0,0.339218,-999.99
1961-06-15,2.9941,3.4137,3.5981,3.6930,3.7501,3.7882,3.8154,NaN,NaN,NaN,...,NaN,3.8694,4.0183,NaN,3.978498,-1.257404,-2.247617,0,0.325775,-999.99
1961-06-16,3.0012,3.4142,3.5994,3.6953,3.7531,3.7917,3.8192,NaN,NaN,NaN,...,NaN,3.8634,4.0242,NaN,3.984350,-1.429538,-1.885024,0,0.348817,-999.99
1961-06-19,2.9949,3.4386,3.6252,3.7199,3.7768,3.8147,3.8418,NaN,NaN,NaN,...,NaN,3.9196,4.0447,NaN,4.004379,-0.723311,-3.310743,0,0.282087,-999.99
1961-06-20,2.9833,3.4101,3.5986,3.6952,3.7533,3.7921,3.8198,NaN,NaN,NaN,...,NaN,3.8732,4.0257,NaN,3.985789,-0.900432,-2.844809,0,0.310316,-999.99


In [113]:
nominal_data.describe()

,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10,...,SVENF30,SVEN1F01,SVEN1F04,SVEN1F09,BETA0,BETA1,BETA2,BETA3,TAU1,TAU2
count,13674.000000,13674.000000,13674.000000,13674.000000,13674.000000,13674.000000,13674.000000,11138.000000,11138.000000,11138.000000,...,7584.000000,13674.000000,13674.000000,11138.000000,13675.000000,13675.000000,13675.000000,13675.000000,13675.000000,13675.000000
mean,5.243493,5.472491,5.656220,5.809632,5.941215,6.055520,6.155301,6.507454,6.598660,6.678448,...,5.454891,5.807921,6.591085,7.550113,5.895122,-0.694114,-353.153208,356.234409,2.399716,-331.983004
std,3.274108,3.191461,3.098593,3.009209,2.928038,2.856445,2.794460,2.921639,2.863796,2.814441,...,2.002320,3.239780,2.763377,2.528758,4.701013,5.083327,5784.740593,5784.459663,3.429810,477.212955
min,0.082800,0.158300,0.302500,0.431300,0.588600,0.762000,0.941900,1.121400,1.295500,1.460700,...,0.411100,0.214300,1.221400,2.477600,0.000000,-39.730118,-340683.769222,-104.029978,0.100000,-999.990000
25%,3.148450,3.462825,3.697200,3.824775,3.936650,4.011350,4.074350,4.320725,4.471000,4.587450,...,4.063375,3.847050,4.422900,5.873500,2.881424,-2.853975,-12.728103,0.000000,0.613517,-999.990000
50%,5.279400,5.466200,5.640650,5.748300,5.855400,5.950150,6.017700,6.455500,6.509650,6.539300,...,4.950700,5.785400,6.346300,7.159950,5.063455,-0.858832,-0.713981,4.854834,1.417131,3.272939
75%,7.091825,7.321200,7.442850,7.561700,7.653225,7.696325,7.717350,8.136275,8.160050,8.188150,...,7.157750,7.705750,8.146200,8.794625,7.949123,1.562920,2.083248,19.714577,2.650038,11.369551
max,16.462000,15.911800,15.574600,15.349800,15.177600,15.061100,15.010900,14.979900,14.957200,14.939800,...,11.502300,16.525200,15.278200,15.743100,25.000000,97.176149,94.872543,340681.502718,30.000000,180.858721


In [115]:
tips_data=tips_data.filter(regex='TIPSY')
nominal_data=nominal_data.filter(regex='SVENY')